In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Get rid of franchised stores

In [4]:
raw_df = pd.read_csv("4_23_cleaned_data.csv")

franchised_stores_df = pd.read_excel("franchised_stores.xlsx")

/var/folders/rc/2n83tm652837dxjbhsw257_m0000gn/T/ipykernel_40207/2488178902.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv("/Users/keon/Desktop/walmart2/dtxwalmart/4_23_cleaned_data.csv")


In [5]:
df_no_franchise = raw_df[~raw_df['Location ID'].isin(franchised_stores_df['Store'])]

## Combine same day pickups

In [6]:

# Group by 'Location ID' and 'Completed Date', summing the 'Tonnage'
grouped_df = df_no_franchise.groupby(['Location ID', 'Completed Date'], as_index=False).agg({
    'Tonnage': 'sum'
})

# Merge the summed 'Tonnage' back to the original dataframe
merged_df = pd.merge(df_no_franchise.drop(columns=['Tonnage']), grouped_df, on=['Location ID', 'Completed Date'])

# Drop duplicates to keep only one row per 'Location ID' and 'Completed Date'
final_df = merged_df.drop_duplicates(subset=['Location ID', 'Completed Date'])


In [7]:
final_df

,Unnamed: 0,Location ID,Location Name,Address,Provider,Trade,Category,ProblemCode,Completed Date,Region,District,Priority,Zip code,City,State,Latitude,Longitude,Tonnage
0,0,3615,Neighborhood Market FM Reg14-12 INSOURCE,2217 NW 23RD ST,Westrock,SOLID WASTE,REPAIR - REACTIVE,Empty and Return (not a repair),2020-11-03,64.0,533.0,NaN,NaN,OKLAHOMA CITY,OK,35.494581,-97.550623,1.51
1,1,830,Walmart FM Reg 12-2,1901 1ST AVE,Rubicon Global,SOLID WASTE,REPAIR - REACTIVE,Empty and Return (not a repair),2020-11-10,25.0,279.0,NaN,NaN,ROCK FALLS,IL,41.763361,-89.687051,2.78
2,3,2733,Supercenter FM Reg 11-2,1009 SAINT PATRICKS DR,Westrock,SOLID WASTE,MAINTENANCE,Compactor is full and needs to be hauled,2020-11-27,2.0,32.0,NaN,NaN,PERRY,GA,32.472626,-83.739099,10.86
3,4,1059,Supercenter FM Reg 15-10 INSOURCE,1309 HIGHWAY 35 S,Westrock,SOLID WASTE,MAINTENANCE,Compactor is full and needs to be hauled,2020-12-01,11.0,138.0,NaN,NaN,FOREST,MS,32.338638,-89.486173,3.88
4,5,2199,Supercenter FM Reg 11-1,50739 VALLEY PLAZA DR,Rubicon Global,SOLID WASTE,MAINTENANCE,Compactor is full and needs to be hauled,2020-12-01,20.0,216.0,NaN,NaN,SAINT CLAIRSVILLE,OH,40.075741,-80.875036,8.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332130,413704,3479,Neighborhood Market,1819 S 8TH ST,NaN,NaN,NaN,NaN,2024-01-31,NaN,NaN,NaN,72756.0,ROGERS,AR,36.313291,-94.124261,2.99
332131,413705,5370,WM Supercenter,2514 MAIN ST,NaN,NaN,NaN,NaN,2024-01-31,NaN,NaN,NaN,80504.0,LONGMONT,CO,40.205153,-105.098807,7.75
332132,413706,4839,Sam's Club,7757 S OLYMPIA AVENUE W,NaN,NaN,NaN,NaN,2024-01-31,NaN,NaN,NaN,74132.0,TULSA,OK,36.050971,-96.003787,9.33
332133,413707,6631,Sam's Club,880 S ABILENE ST,NaN,NaN,NaN,NaN,2024-01-31,NaN,NaN,NaN,80012.0,AURORA,CO,39.700498,-104.825655,7.63


## Turn dates into datetime, filter for dates between Feb 2021 and Aug 2023

In [8]:
type(final_df.loc[0, 'Completed Date'])

str

In [9]:
final_df['Completed Date'] = pd.to_datetime(final_df['Completed Date'])

# Define the date range
start_date = '2021-02-01'
end_date = '2023-09-01'

# Filter the DataFrame for the date range
filtered_df = final_df[(final_df['Completed Date'] >= start_date) & (final_df['Completed Date'] <= end_date)]

/var/folders/rc/2n83tm652837dxjbhsw257_m0000gn/T/ipykernel_40207/177580384.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Completed Date'] = pd.to_datetime(final_df['Completed Date'])


## Drop Irrelevant Columns

In [10]:
df = filtered_df[["Location ID", "Tonnage", "Completed Date", "Zip code", "State", "Latitude", "Longitude"]]

## Create Lagging Indicators

In [11]:
# Sort the DataFrame
df.sort_values(by=['Location ID', 'Completed Date'], inplace=True)

/var/folders/rc/2n83tm652837dxjbhsw257_m0000gn/T/ipykernel_40207/1444988040.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by=['Location ID', 'Completed Date'], inplace=True)


In [12]:
# Calculate the number of days between current and last pickup
df['Days Since Last Pickup'] = df.groupby('Location ID')['Completed Date'].diff().dt.days

/var/folders/rc/2n83tm652837dxjbhsw257_m0000gn/T/ipykernel_40207/1293491196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Days Since Last Pickup'] = df.groupby('Location ID')['Completed Date'].diff().dt.days


In [13]:
# Extract information about the last three pickups
df['Last Pickup Tonnage'] = df.groupby('Location ID')['Tonnage'].shift(1)
df['Second Last Pickup Tonnage'] = df.groupby('Location ID')['Tonnage'].shift(2)
df['Third Last Pickup Tonnage'] = df.groupby('Location ID')['Tonnage'].shift(3)

/var/folders/rc/2n83tm652837dxjbhsw257_m0000gn/T/ipykernel_40207/1317808763.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Last Pickup Tonnage'] = df.groupby('Location ID')['Tonnage'].shift(1)
/var/folders/rc/2n83tm652837dxjbhsw257_m0000gn/T/ipykernel_40207/1317808763.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Second Last Pickup Tonnage'] = df.groupby('Location ID')['Tonnage'].shift(2)
/var/folders/rc/2n83tm652837dxjbhsw257_m0000gn/T/ipykernel_40207/1317808763.py:4: SettingWithCopyWarn

In [14]:
weekmask = '1111100'  # Monday=1, Tuesday=1, Wednesday=1, Thursday=1, Friday=1, Saturday=0, Sunday=0

def calculate_weekdays(row):
    if pd.isna(row['Days Since Last Pickup']):
        return np.nan
    start_date = row['Completed Date'] - pd.to_timedelta(row['Days Since Last Pickup'], unit='d')
    end_date = row['Completed Date']
    return np.busday_count(start_date.date(), end_date.date(), weekmask=weekmask)

df['Weekdays Since Last Pickup'] = df.apply(calculate_weekdays, axis=1)

/var/folders/rc/2n83tm652837dxjbhsw257_m0000gn/T/ipykernel_40207/3490519654.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Weekdays Since Last Pickup'] = df.apply(calculate_weekdays, axis=1)


In [15]:
weekendmask = '0000111'
def calculate_weekends(row):
    if pd.isna(row['Days Since Last Pickup']):
        return np.nan
    start_date = row['Completed Date'] - pd.to_timedelta(row['Days Since Last Pickup'], unit='d')
    end_date = row['Completed Date']
    return np.busday_count(start_date.date(), end_date.date(), weekmask=weekendmask)

df['Weekend Days Since Last Pickup'] = df.apply(calculate_weekends, axis = 1)

/var/folders/rc/2n83tm652837dxjbhsw257_m0000gn/T/ipykernel_40207/4251129429.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Weekend Days Since Last Pickup'] = df.apply(calculate_weekends, axis = 1)


In [16]:
df['Days Before Last Pickup'] = df.groupby('Location ID')['Days Since Last Pickup'].shift(1)
df['Days Before Second Last Pickup'] = df.groupby('Location ID')['Days Since Last Pickup'].shift(2)
df['Days Before Third Last Pickup'] = df.groupby('Location ID')['Days Since Last Pickup'].shift(3)

/var/folders/rc/2n83tm652837dxjbhsw257_m0000gn/T/ipykernel_40207/3768080695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Days Before Last Pickup'] = df.groupby('Location ID')['Days Since Last Pickup'].shift(1)
/var/folders/rc/2n83tm652837dxjbhsw257_m0000gn/T/ipykernel_40207/3768080695.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Days Before Second Last Pickup'] = df.groupby('Location ID')['Days Since Last Pickup'].shift(2)
/var/folders/rc/2n83tm652837dxjbhsw257_m0000gn/T/ipykernel_4020

## Create Month Variable

In [17]:
df['Month'] = df['Completed Date'].dt.month

/var/folders/rc/2n83tm652837dxjbhsw257_m0000gn/T/ipykernel_40207/3978473839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month'] = df['Completed Date'].dt.month


In [18]:
df

,Location ID,Tonnage,Completed Date,Zip code,State,Latitude,Longitude,Days Since Last Pickup,Last Pickup Tonnage,Second Last Pickup Tonnage,Third Last Pickup Tonnage,Weekdays Since Last Pickup,Weekend Days Since Last Pickup,Days Before Last Pickup,Days Before Second Last Pickup,Days Before Third Last Pickup,Month
8235,1,8.90,2021-02-05,NaN,AR,36.331300,-94.149054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
9020,1,9.15,2021-02-08,72756.0,AR,36.331300,-94.149054,3.0,8.90,NaN,NaN,1.0,3.0,NaN,NaN,NaN,2
11492,1,5.32,2021-02-19,72756.0,AR,36.331300,-94.149054,11.0,9.15,8.90,NaN,9.0,3.0,3.0,NaN,NaN,2
13478,1,5.61,2021-02-26,72756.0,AR,36.331300,-94.149054,7.0,5.32,9.15,8.90,5.0,3.0,11.0,3.0,NaN,2
15892,1,6.69,2021-03-08,72756.0,AR,36.331300,-94.149054,10.0,5.61,5.32,9.15,6.0,6.0,7.0,11.0,3.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257575,30095,13.38,2023-05-26,72712.0,AR,36.346243,-94.195376,120.0,12.62,11.31,3.99,86.0,51.0,115.0,224.0,173.0,5
262255,30095,1.01,2023-06-09,72712.0,AR,36.346243,-94.195376,14.0,13.38,12.62,11.31,10.0,6.0,120.0,115.0,224.0,6
146562,30099,10.79,2022-06-06,72712.0,AR,36.340990,-94.200230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
200247,30099,7.46,2022-11-25,72712.0,AR,36.340990,-94.200230,172.0,10.79,NaN,NaN,124.0,72.0,NaN,NaN,NaN,11


In [19]:
df[df["Days Since Last Pickup"] == 0]

,Location ID,Tonnage,Completed Date,Zip code,State,Latitude,Longitude,Days Since Last Pickup,Last Pickup Tonnage,Second Last Pickup Tonnage,Third Last Pickup Tonnage,Weekdays Since Last Pickup,Weekend Days Since Last Pickup,Days Before Last Pickup,Days Before Second Last Pickup,Days Before Third Last Pickup,Month


 ## Drop 0 Tonnage Pickups

In [24]:
df = df[df['Tonnage'] != 0]

In [25]:
df.to_csv("processed_data.csv", index=False)